# COMS 4995_002 Deep Learning Assignment 2

The UNIs of our team

Member 1: Saahil Jain, sj2675

Member 2: Leon Stilwell, ls3223

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.misc
import glob
import sys
import tensorflow as tf

//anaconda/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


In [2]:
np.random.seed(1)

In [3]:
# helper functions
def get_img_array(path):
    """
    Given path of image, returns it's numpy array
    """
    return scipy.misc.imread(path)

def get_files(folder):
    """
    Given path to folder, returns list of files in it
    """
    filenames = [file for file in glob.glob(folder+'*/*')]
    filenames.sort()
    return filenames

def get_label(filepath, label2id):
    """
    Files are assumed to be labeled as: /path/to/file/999_frog.png
    Returns label for a filepath
    """
    tokens = filepath.split('/')
    label = tokens[-1].split('_')[1][:-4]
    if label in label2id:
        return label2id[label]
    else:
        sys.exit("Invalid label: " + label)

In [4]:
#functions to load data

def get_labels(folder, label2id):
    """
    Returns vector of labels extracted from filenames of all files in folder
    :param folder: path to data folder
    :param label2id: mapping of text labels to numeric ids. (Eg: automobile -> 0)
    """
    files = get_files(folder)
    y = []
    for f in files:
        y.append(get_label(f,label2id))
    return np.array(y)

def one_hot(y, num_classes=10):
    """
    Converts each label index in y to vector with one_hot encoding
    """
    y_one_hot = np.zeros((num_classes, y.shape[0]))
    y_one_hot[y, range(y.shape[0])] = 1
    return y_one_hot

def get_label_mapping(label_file):
    """
    Returns mappings of label to index and index to label
    The input file has list of labels, each on a separate line.
    """
    with open(label_file, 'r') as f:
        id2label = f.readlines()
        id2label = [l.strip() for l in id2label]
    label2id = {}
    count = 0
    for label in id2label:
        label2id[label] = count
        count += 1
    return id2label, label2id

def get_images(folder, testFile = False):
    """
    returns numpy array of all samples in folder
    each column is a sample resized to 30x30 and flattened
    """
    files = get_files(folder)
    if (testFile):
        files.sort(key = lambda x: int(x.split("/")[2].split(".")[0]))
    images = []
    count = 0
    
    for f in files:
        count += 1
        if count % 10000 == 0:
            print("Loaded {}/{}".format(count,len(files)))
        img_arr = get_img_array(f)
        img_arr = img_arr.flatten() / 255.0
        images.append(img_arr)
    X = np.column_stack(images)

    return X

def get_train_data(data_root_path):
    """
    Return X and y
    """
    train_data_path = data_root_path + 'train'
    id2label, label2id = get_label_mapping(data_root_path+'labels.txt')
    print(label2id)
    X = get_images(train_data_path)
    y = get_labels(train_data_path, label2id)
    return X, y

def save_predictions(filename, y):
    """
    Dumps y into .npy file
    """
    np.save(filename, y)

# Extract / format data

In [5]:
data_root_path = "cifar10-hw2/"
X_train, y_train = get_train_data(data_root_path)
X_test = get_images(data_root_path + 'test', True)
print('Data loading done')

{'automobile': 1, 'horse': 7, 'airplane': 0, 'ship': 8, 'deer': 4, 'frog': 6, 'cat': 3, 'bird': 2, 'truck': 9, 'dog': 5}
Loaded 10000/50000
Loaded 20000/50000
Loaded 30000/50000
Loaded 40000/50000
Loaded 50000/50000
Loaded 10000/10000
Data loading done


In [6]:
# Reshape data as necessary
y_train.reshape(50000, )
X_train = np.swapaxes(X_train,1,0)
X_test = np.swapaxes(X_test,1,0)

# Convert to Numpy array
X_train = np.asarray(X_train,dtype = np.float32)
X_test = np.asarray(X_test,dtype = np.float32)
y_train = np.asarray(y_train,dtype = np.int32)

# Print shapes
print("X_train shape is: " + str(X_train.shape))
print("X_test shape is: " + str(X_test.shape))
print("y_train shape is: " + str(y_train.shape))

X_train shape is: (50000, 3072)
X_test shape is: (10000, 3072)
y_train shape is: (50000,)


In [7]:
# Shuffle training data
shuffledIndices = np.arange(len(X_train))
np.random.shuffle(shuffledIndices)
X_train = X_train[shuffledIndices]
y_train = y_train[shuffledIndices]

# Split into train and validation data
X_validation = X_train[-5000:,:]
X_train = X_train[:45000]
y_validation = y_train[-5000:]
y_train = y_train[:45000]

# Print shapes
print(X_train.shape)
print(X_validation.shape)
print(y_validation.shape)

(45000, 3072)
(5000, 3072)
(5000,)


# Tensorflow Models

In [13]:
tf.logging.set_verbosity(tf.logging.INFO)
LOGDIR = "tmp/cifar-10-deep/"

In [14]:
tf.reset_default_graph()
def my_model_fn(features, labels, mode, params): 
    batch = tf.reshape(features["x"],[-1,32,32,3])
    tf.summary.image("image",batch,3)
    height = 24
    width = 24
    if(mode == tf.estimator.ModeKeys.TRAIN):
        unpacked = tf.unstack(batch)
        processed = []
        with tf.name_scope('data_augmentation'):
            for i in unpacked:
                distorted_image = tf.random_crop(i,[height,width,3])
                distorted_image = tf.image.random_flip_left_right(distorted_image)
                distorted_image = tf.image.random_brightness(distorted_image,max_delta=63)
                distorted_image = tf.image.random_contrast(distorted_image,lower=.2,upper=1.8)
                # Subtract off the mean and divide by the variance of the pixels.
                float_image = tf.image.per_image_standardization(distorted_image)
                float_image.set_shape([height, width, 3])
                processed.append(float_image)
        input_layer = tf.reshape(processed,[-1,24,24,3])
        tf.summary.image("augmented-image",input_layer,3)
    else:
        batch = tf.reshape(batch,[1,32,32,3])
        images = tf.unstack(batch)
        processed = []
        for i in images:
            #image = tf.reshape(i,[32,32,3])
            resized_image = tf.image.resize_image_with_crop_or_pad(i,24,24)
            float_image = tf.image.per_image_standardization(resized_image)
            float_image.set_shape([24, 24, 3])
            processed.append(float_image)
        input_layer = tf.reshape(processed,[-1,24,24,3])
   
    # Convolution Layer #1
    with tf.name_scope("conv1"):
        conv1 = tf.layers.conv2d(
          inputs=input_layer,
          filters=64,
          kernel_size=[5,5],
          padding="same",
          activation=tf.nn.relu,
          name = "conv1"
          )
        tf.summary.histogram("conv1",conv1)
        
    # Pooling Layer #1
    with tf.name_scope("pool1"):
        pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size = [2,2],strides = 2,name="pool1")
        tf.summary.histogram("pool1",pool1)
        
    # Convolutional Layer #2
    with tf.name_scope("conv2"):
        conv2 = tf.layers.conv2d(inputs = pool1, filters = 64, kernel_size = [5,5],
                                 padding = 'same', activation = tf.nn.relu,name="conv2")
        tf.summary.histogram("conv2",conv2)
            
    # Pooling Layer #2
    with tf.name_scope("pool2"):
        pool2 = tf.layers.max_pooling2d(inputs = conv2, pool_size=[2,2],strides = 2,name="pool2")
        tf.summary.histogram("pool2",pool2)
        
    # Convolutional Layer #3
    with tf.name_scope("conv3"):
        conv3 = tf.layers.conv2d(inputs = pool2, filters = 128, kernel_size = [3,3],
                                 padding = 'same', activation = tf.nn.relu,name="conv3")
        tf.summary.histogram("conv3",conv3)
        
    # Convolutional Layer #4
    with tf.name_scope("conv4"):
        conv4 = tf.layers.conv2d(inputs = conv3, filters = 128, kernel_size = [3,3],
                                 padding = 'same', activation = tf.nn.relu,name="conv4")
        tf.summary.histogram("conv4",conv4)
        
    # Convolutional Layer #5
    with tf.name_scope("conv5"):
        conv5 = tf.layers.conv2d(inputs = conv4, filters = 128, kernel_size = [3,3],
                                 padding = 'same', activation = tf.nn.relu,name="conv5")
        tf.summary.histogram("conv5",conv5)
        
    # Pooling Layer #3
    with tf.name_scope("pool3"):
        pool3 = tf.layers.max_pooling2d(inputs = conv5, pool_size=[2,2],strides = 2,name="pool3")
        tf.summary.histogram("pool3",pool3)
    
    # Flatten for dense layer
    with tf.name_scope("pool3_flat"):
        pool3_flat = tf.reshape(pool3, [-1, 3*3*128])
        tf.summary.histogram("pool3_flat",pool3_flat)
    
    # Dense 1 Layer
    with tf.name_scope("dense1"):
        dense1 = tf.layers.dense(inputs=pool3_flat, units=400, activation=tf.nn.relu,name="dense1")
        tf.summary.histogram("dense1",dense1)
        
    # Dense 2 Layer
    with tf.name_scope("dense2"):
        dense2 = tf.layers.dense(inputs=dense1, units=200, activation=tf.nn.relu,name="dense2")
        tf.summary.histogram("dense2",dense2)
    
    # Dropout Layer
    with tf.name_scope("dropout"):
        dropout = tf.layers.dropout(
            inputs=dense2, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN,name="dropout")
        tf.summary.histogram("dropout",dropout)
    
    # Logits Layer
    with tf.name_scope("logits"):
        logits = tf.layers.dense(inputs = dropout, units = 10,name="logits")
        tf.summary.histogram("logits",logits)
    
    # Make predictions
    predictions = {
        "classes" : tf.argmax(input=logits, axis=1),
        "probabilities" : tf.nn.softmax(logits,name = "softmax_tensor")
        }
    
    if(mode == tf.estimator.ModeKeys.PREDICT):
        return tf.estimator.EstimatorSpec(mode = mode, predictions=predictions)

    with tf.name_scope("train-accuracy"):
        train_prediction = tf.nn.softmax(logits)
        cast_labels = tf.cast(labels,tf.int64)
        correct_prediction = tf.equal(tf.argmax(train_prediction, 1), cast_labels)
      
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        tf.summary.scalar("train-accuracy", accuracy)
        
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    #l2_loss = tf.losses.get_regularization_loss()
    #loss += l2_loss
    
    logging_hook = tf.train.LoggingTensorHook({"training accuracy" : accuracy}, 
                                              every_n_iter=100)

    if mode == tf.estimator.ModeKeys.EVAL:
        eval_metric_ops = {
            "accuracy": tf.metrics.accuracy(
                labels=labels, predictions=predictions["classes"])}
        return tf.estimator.EstimatorSpec(
            mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
        # Display gradients in Tensorboard
        gradients = optimizer.compute_gradients(loss)
        for index, grad in enumerate(gradients):
            tf.summary.histogram("{}-grad".format(gradients[index][1].name), gradients[index])
        # Optimize
        train_op = optimizer.minimize(loss = loss, global_step = tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op, training_hooks = [logging_hook])
    
    summ = tf.summary.merge_all()
    writer = tf.summary.FileWriter(LOGDIR)
    writer.add_graph(sess.graph)

In [15]:
feature_columns = [tf.feature_column.numeric_column("x", shape=[3072], dtype=tf.float32)]
tf.reset_default_graph()

# Classifier
classifier = tf.estimator.Estimator(
    model_fn = my_model_fn,
    model_dir = LOGDIR,
    params={
        'feature_columns' : feature_columns,
        #choose between 10 classes
        'n_classes':10,
    }
)

train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x = {"x": X_train},
    y = y_train,
    batch_size = 100,
    num_epochs = None,
    shuffle = True
    )

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_keep_checkpoint_max': 5, '_log_step_count_steps': 100, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x13332e128>, '_tf_random_seed': None, '_session_config': None, '_service': None, '_task_type': 'worker', '_save_summary_steps': 100, '_task_id': 0, '_model_dir': 'tmp/cifar-10-deep/', '_num_worker_replicas': 1, '_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_is_chief': True, '_save_checkpoints_steps': None, '_master': '', '_keep_checkpoint_every_n_hours': 10000}


In [ ]:
classifier.train(input_fn=train_input_fn, steps=50000)

INFO:tensorflow:Summary name conv1/kernel:0-grad is illegal; using conv1/kernel_0-grad instead.
INFO:tensorflow:Summary name conv1/bias:0-grad is illegal; using conv1/bias_0-grad instead.
INFO:tensorflow:Summary name conv2/kernel:0-grad is illegal; using conv2/kernel_0-grad instead.
INFO:tensorflow:Summary name conv2/bias:0-grad is illegal; using conv2/bias_0-grad instead.
INFO:tensorflow:Summary name conv3/kernel:0-grad is illegal; using conv3/kernel_0-grad instead.
INFO:tensorflow:Summary name conv3/bias:0-grad is illegal; using conv3/bias_0-grad instead.
INFO:tensorflow:Summary name conv4/kernel:0-grad is illegal; using conv4/kernel_0-grad instead.
INFO:tensorflow:Summary name conv4/bias:0-grad is illegal; using conv4/bias_0-grad instead.
INFO:tensorflow:Summary name conv5/kernel:0-grad is illegal; using conv5/kernel_0-grad instead.
INFO:tensorflow:Summary name conv5/bias:0-grad is illegal; using conv5/bias_0-grad instead.
INFO:tensorflow:Summary name dense1/kernel:0-grad is illegal

## Test Validation Data

In [26]:
# Evaluate the model and print results
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
  x={"x": X_validation},
  y=y_validation,
  batch_size = 1,
  shuffle=False)
eval_results = classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

INFO:tensorflow:Starting evaluation at 2018-02-27-16:13:47
INFO:tensorflow:Restoring parameters from tmp/cifar-10-2fc-384-192-64-filters-norm/model.ckpt-55653
INFO:tensorflow:Finished evaluation at 2018-02-27-16:14:01
INFO:tensorflow:Saving dict for global step 55653: accuracy = 0.8196, global_step = 55653, loss = 0.5922929
{'global_step': 55653, 'accuracy': 0.8196, 'loss': 0.5922929}


## Predict Test Data

In [22]:
# Evaluate the model and print results
predict_input_fn = tf.estimator.inputs.numpy_input_fn(
  x={"x": X_test},
  num_epochs=1,
  batch_size = 1,
  shuffle=False)
predict_results = classifier.predict(input_fn=predict_input_fn)
predictions = list(predict_results)

INFO:tensorflow:Restoring parameters from tmp/cifar-10-2fc-384-192-64-filters-norm/model.ckpt-55653


In [23]:
y_predicted = []
for pred in predictions:
    y_predicted.append(pred['classes'])
y_predicted = np.array(y_predicted)

## Save Predictions, Create Submission File

In [27]:
save_predictions('ans-sj2675.npy', y_predicted)

In [28]:
# test if numpy file has been saved correctly
loaded_y = np.load('ans-sj2675.npy')
print(loaded_y.shape)
loaded_y[:10]

(10000,)


array([3, 0, 8, 0, 6, 6, 5, 6, 3, 1])

In [29]:
import csv

In [30]:
def createSubmissionFile(predictions):
    with open('test_submit.csv', 'w') as csvfile:
        fieldnames = ['ID', 'CLASS']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for ID in range(len(predictions)):
            writer.writerow({'ID': ID, 'CLASS': predictions[ID]})

In [31]:
createSubmissionFile(y_predicted)

# To Delete Later

In [32]:
labels = [3, 8, 8, 0, 6, 6, 1, 6, 3, 1, 0, 9, 5, 7, 9, 8, 5, 7, 8, 6, 7, 0, 4, 9, 5, 2, 4, 0, 9, 6, 6, 5, 4, 5, 9, 2, 4, 1, 9, 5, 4, 6, 5, 6, 0, 9, 3, 9, 7, 6, 9, 8, 0, 3, 8, 8, 7, 7, 4, 6, 7, 3, 6, 3, 6, 2, 1, 2, 3, 7, 2, 6, 8, 8, 0, 2, 9, 3, 3, 8, 8, 1, 1, 7, 2, 5, 2, 7, 8, 9, 0, 3, 8, 6, 4, 6, 6, 0, 0, 7, 4, 5, 6, 3, 1, 1, 3, 6, 8, 7, 4, 0, 6, 2, 1, 3, 0, 4, 2, 7, 8, 3, 1, 2, 8, 0, 8, 3, 5, 2, 4, 1, 8, 9, 1, 2, 9, 7, 2, 9, 6, 5, 6, 3, 8, 7, 6, 2, 5, 2, 8, 9, 6, 0, 0, 5, 2, 9, 5, 4, 2, 1, 6, 6, 8, 4, 8, 4, 5, 0, 9, 9, 9, 8, 9, 9, 3, 7, 5, 0, 0, 5, 2, 2, 3, 8, 6, 3, 4, 0, 5, 8, 0, 1, 7, 2, 8, 8, 7, 8, 5, 1, 8, 7, 1, 3, 0, 5, 7, 9, 7, 4, 5, 9, 8, 0, 7, 9, 8, 2, 7, 6, 9, 4, 3, 9, 6, 4, 7, 6, 5, 1, 5, 8, 8, 0, 4, 0, 5, 5, 1, 1, 8, 9, 0, 3, 1, 9, 2, 2, 5, 3, 9, 9, 4, 0, 3, 0, 0, 9, 8, 1, 5, 7, 0, 8, 2, 4, 7, 0, 2, 3, 6, 3, 8, 5, 0, 3, 4, 3, 9, 0, 6, 1, 0, 9, 1, 0, 7, 9, 1, 2, 6, 9, 3, 4, 6, 0, 0, 6, 6, 6, 3, 2, 6, 1, 8, 2, 1, 6, 8, 6, 8, 0, 4, 0, 7, 7, 5, 5, 3, 5, 2, 3, 4, 1, 7, 5, 4, 6, 1, 9, 3, 6, 6, 9, 3, 8, 0, 7, 2, 6, 2, 5, 8, 5, 4, 6, 8, 9, 9, 1, 0, 2, 2, 7, 3, 2, 8, 0, 9, 5, 8, 1, 9, 4, 1, 3, 8, 1, 4, 7, 9, 4, 2, 7, 0, 7, 0, 6, 6, 9, 0, 9, 2, 8, 7, 2, 2, 5, 1, 2, 6, 2, 9, 6, 2, 3, 0, 3, 9, 8, 7, 8, 8, 4, 0, 1, 8, 2, 7, 9, 3, 6, 1, 9, 0, 7, 3, 7, 4, 5, 0, 0, 2, 9, 3, 4, 0, 6, 2, 5, 3, 7, 3, 7, 2, 5, 3, 1, 1, 4, 9, 9, 5, 7, 5, 0, 2, 2, 2, 9, 7, 3, 9, 4, 3, 5, 4, 6, 5, 6, 1, 4, 3, 4, 4, 3, 7, 8, 3, 7, 8, 0, 5, 7, 6, 0, 5, 4, 8, 6, 8, 5, 5, 9, 9, 9, 5, 0, 1, 0, 8, 1, 1, 8, 0, 2, 2, 0, 4, 6, 5, 4, 9, 4, 7, 9, 9, 4, 5, 6, 6, 1, 5, 3, 8, 9, 5, 8, 5, 7, 0, 7, 0, 5, 0, 0, 4, 6, 9, 0, 9, 5, 6, 6, 6, 2, 9, 0, 1, 7, 6, 7, 5, 9, 1, 6, 2, 5, 5, 5, 8, 5, 9, 4, 6, 4, 3, 2, 0, 7, 6, 2, 2, 3, 9, 7, 9, 2, 6, 7, 1, 3, 6, 6, 8, 9, 7, 5, 4, 0, 8, 4, 0, 9, 3, 4, 8, 9, 6, 9, 2, 6, 1, 4, 7, 3, 5, 3, 8, 5, 0, 2, 1, 6, 4, 3, 3, 9, 6, 9, 8, 8, 5, 8, 6, 6, 2, 1, 7, 7, 1, 2, 7, 9, 9, 4, 4, 1, 2, 5, 6, 8, 7, 6, 8, 3, 0, 5, 5, 3, 0, 7, 9, 1, 3, 4, 4, 5, 3, 9, 5, 6, 9, 2, 1, 1, 4, 1, 9, 4, 7, 6, 3, 8, 9, 0, 1, 3, 6, 3, 6, 3, 2, 0, 3, 1, 0, 5, 9, 6, 4, 8, 9, 6, 9, 6, 3, 0, 3, 2, 2, 7, 8, 3, 8, 2, 7, 5, 7, 2, 4, 8, 7, 4, 2, 9, 8, 8, 6, 8, 8, 7, 4, 3, 3, 8, 4, 9, 4, 8, 8, 1, 8, 2, 1, 3, 6, 5, 4, 2, 7, 9, 9, 4, 1, 4, 1, 3, 2, 7, 0, 7, 9, 7, 6, 6, 2, 5, 9, 2, 9, 1, 2, 2, 6, 8, 2, 1, 3, 6, 6, 0, 1, 2, 7, 0, 5, 4, 6, 1, 6, 4, 0, 2, 2, 6, 0, 5, 9, 1, 7, 6, 7, 0, 3, 9, 6, 8, 3, 0, 3, 4, 7, 7, 1, 4, 7, 2, 7, 1, 4, 7, 4, 4, 8, 4, 7, 7, 5, 3, 7, 2, 0, 8, 9, 5, 8, 3, 6, 2, 0, 8, 7, 3, 7, 6, 5, 3, 1, 3, 2, 2, 5, 4, 1, 2, 9, 2, 7, 0, 7, 2, 1, 3, 2, 0, 2, 4, 7, 9, 8, 9, 0, 7, 7, 0, 7, 8, 4, 6, 3, 3, 0, 1, 3, 7, 0, 1, 3, 1, 4, 2, 3, 8, 4, 2, 3, 7, 8, 4, 3, 0, 9, 0, 0, 1, 0, 4, 4, 6, 7, 6, 1, 1, 3, 7, 3, 5, 2, 6, 6, 5, 8, 7, 1, 6, 8, 8, 5, 3, 0, 4, 0, 1, 3, 8, 8, 0, 6, 9, 9, 9, 5, 5, 8, 6, 0, 0, 4, 2, 3, 2, 7, 2, 2, 5, 9, 8, 9, 1, 7, 4, 0, 3, 0, 1, 3, 8, 3, 9, 6, 1, 4, 7, 0, 3, 7, 8, 9, 1, 1, 6, 6, 6, 6, 9, 1, 9, 9, 4, 2, 1, 7, 0, 6, 8, 1, 9, 2, 9, 0, 4, 7, 8, 3, 1, 2, 0, 1, 5, 8, 4, 6, 3, 8, 1, 3, 8, 5, 0, 8, 4, 8, 1, 1, 8, 9, 6, 0, 8, 6, 1, 3, 4, 1, 6, 0, 5, 1, 1, 0, 0, 3, 5, 0, 0, 6, 6, 3, 3, 6, 3, 6, 6, 0, 7, 2, 2, 7, 5, 5, 2, 8, 5, 2, 1, 1, 4, 3, 2, 0, 3, 1, 5, 3, 7, 6, 8, 9, 1, 6, 4, 9, 3, 9, 0, 9, 6, 3, 6, 0, 7, 3, 8, 0, 0, 0, 6, 6, 6, 9, 2, 5, 4, 4, 6, 3, 6, 0, 8, 6, 0, 6, 2, 7, 5, 1, 2, 7, 8, 8, 0, 9, 4, 9, 7, 2, 0, 2, 8, 3, 8, 9, 1, 5, 5, 4, 7, 5, 3, 8, 3, 3, 6, 2, 8, 4, 3, 7, 1, 2, 4, 1, 6, 9, 0, 5, 8, 6, 1, 8, 6, 1, 4, 2, 6, 2, 7, 2, 2, 0, 8, 6, 9, 1, 7, 1, 8, 8, 0, 7, 3, 8, 0, 3, 4, 3, 7, 7, 9, 2, 3, 1, 9, 1, 9, 6, 3, 3, 3, 1, 0, 6, 1, 4, 1, 0, 0, 1, 1, 6, 5, 4, 6, 2, 0, 7, 9, 8, 7, 2, 0, 6, 8, 1, 4, 3, 7, 0, 6, 1, 8, 5, 7, 8, 4, 8, 3, 9, 9, 9, 8, 7, 6, 6, 3, 5, 1, 5, 9, 1, 4, 1, 5, 7, 0, 1, 5, 2, 0, 8, 8, 5, 6, 7, 3, 2, 4, 7, 2, 5, 8, 2, 4, 9, 2, 1, 8, 1, 9, 8, 8, 8, 9, 0, 4, 3, 3, 1, 8, 4, 6, 3, 3, 5, 2, 2, 8, 3, 8, 9, 5, 8, 9, 8, 9, 1, 6, 5, 9, 4, 4, 8, 0, 7, 2, 9, 7, 4, 1, 6, 4, 4, 9, 1, 2, 5, 6, 0, 8, 6, 1, 9, 4, 5, 9, 5, 0, 7, 2, 0, 0, 4, 2, 6, 6, 5, 5, 2, 8, 1, 7, 3, 1, 4, 5, 6, 5, 1, 4, 7, 0, 9, 4, 3, 8, 2, 8, 4, 7, 2, 3, 1, 5, 2, 9, 8, 9, 7, 9, 5, 1, 4, 0, 8, 2, 3, 8, 9, 1, 1, 3, 2, 4, 9, 3, 1, 7, 4, 6, 2, 8, 9, 5, 3, 9, 5, 5, 6, 7, 2, 4, 6, 3, 1, 0, 7, 2, 5, 4, 7, 6, 1, 1, 9, 8, 1, 0, 1, 3, 1, 1, 1, 7, 3, 9, 6, 8, 4, 6, 8, 4, 9, 4, 7, 9, 7, 6, 8, 4, 9, 7, 0, 1, 6, 1, 5, 9, 0, 4, 3, 4, 1, 3, 0, 8, 4, 6, 2, 2, 6, 5, 3, 6, 2, 1, 1, 8, 6, 0, 4, 0, 1, 9, 7, 1, 3, 7, 7, 8, 7, 7, 3, 9, 7, 7, 7, 2, 1, 2, 8, 6, 4, 0, 7, 9, 8, 6, 8, 4, 9, 1, 7, 2, 2, 8, 5, 8, 1, 2, 2, 4, 1, 2, 5, 2, 8, 1, 8, 1, 8, 6, 0, 2, 4, 1, 3, 6, 7, 7, 4, 4, 3, 3, 4, 5, 2, 4, 3, 7, 8, 4, 4, 4, 5, 4, 3, 2, 8, 4, 5, 5, 4, 1, 4, 2, 5, 1, 6, 4, 3, 4, 4, 0, 8, 8, 4, 5, 7, 5, 6, 9, 1, 6, 7, 2, 0, 1, 4, 5, 6, 0, 0, 2, 7, 5, 6, 0, 6, 2, 9, 1, 7, 7, 5, 2, 5, 6, 4, 1, 4, 3, 3, 3, 0, 3, 5, 5, 8, 9, 7, 3, 1, 3, 3, 3, 4, 4, 2, 3, 3, 8, 1, 7, 7, 0, 7, 4, 5, 1, 4, 2, 4, 3, 9, 9, 4, 9, 9, 1, 8, 1, 6, 7, 5, 5, 4, 9, 7, 6, 5, 9, 2, 4, 0, 7, 8, 5, 5, 0, 0, 9, 9, 8, 2, 5, 4, 8, 3, 6, 3, 6, 0, 6, 6, 6, 9, 6, 6, 8, 6, 2, 4, 5, 8, 1, 2, 7, 6, 5, 7, 8, 1, 8, 0, 8, 6, 9, 2, 8, 9, 4, 0, 9, 4, 9, 5, 7, 5, 5, 9, 5, 3, 0, 1, 9, 7, 2, 4, 1, 0, 8, 0, 3, 1, 7, 0, 0, 4, 8, 6, 2, 4, 0, 0, 9, 0, 8, 4, 5, 9, 3, 9, 0, 5, 6, 5, 0, 1, 4, 8, 1, 0, 5, 2, 1, 0, 2, 8, 1, 5, 6, 7, 7, 2, 6, 2, 5, 0, 1, 4, 2, 5, 4, 6, 2, 2, 1, 7, 2, 8, 5, 5, 3, 0, 4, 8, 3, 7, 6, 3, 8, 1, 0, 1, 3, 3, 0, 7, 4, 9, 5, 3, 6, 0, 1, 4, 4, 4, 4, 2, 2, 5, 8, 1, 5, 9, 8, 1, 1, 5, 3, 9, 9, 7, 6, 5, 0, 8, 4, 7, 0, 9, 2, 8, 4, 7, 1, 3, 9, 6, 8, 9, 0, 4, 9, 6, 7, 8, 9, 4, 8, 9, 7, 2, 5, 3, 7, 1, 0, 2, 9, 5, 5, 8, 5, 4, 2, 8, 3, 5, 5, 7, 7, 8, 6, 2, 8, 2, 3, 5, 6, 8, 0, 2, 3, 7, 0, 1, 9, 1, 3, 7, 5, 8, 3, 2, 9, 6, 8, 6, 9, 3, 8, 9, 8, 0, 7, 8, 5, 0, 0, 1, 3, 9, 1, 5, 3, 4, 4, 0, 9, 9, 9, 9, 8, 2, 4, 2, 2, 5, 1, 9, 1, 0, 9, 4, 2, 1, 6, 0, 3, 7, 6, 3, 1, 8, 6, 5, 7, 2, 8, 4, 4, 8, 3, 5, 0, 5, 7, 4, 4, 2, 2, 7, 3, 6, 0, 2, 7, 6, 2, 3, 0, 7, 7, 8, 1, 1, 4, 6, 0, 6, 6, 5, 5, 6, 3, 9, 3, 6, 8, 7, 6, 4, 9, 5, 6, 4, 1, 6, 3, 8, 2, 3, 9, 8, 5, 1, 5, 4, 5, 7, 5, 7, 8, 9, 1, 7, 2, 5, 6, 8, 4, 6, 5, 3, 9, 9, 8, 5, 5, 6, 4, 5, 9, 7, 3, 4, 1, 4, 2, 3, 6, 5, 5, 2, 8, 0, 0, 1, 8, 3, 1, 3, 5, 8, 3, 8, 5, 8, 6, 3, 5, 5, 5, 0, 9, 5, 5, 7, 1, 8, 2, 2, 7, 3, 4, 2, 9, 6, 2, 2, 4, 3, 0, 2, 0, 1, 3, 2, 2, 7, 0, 1, 0, 7, 7, 2, 0, 1, 6, 5, 0, 2, 2, 0, 1, 2, 6, 0, 1, 6, 6, 5, 3, 4, 0, 0, 9, 1, 0, 2, 5, 9, 7, 8, 6, 4, 6, 0, 2, 0, 1, 9, 2, 4, 9, 0, 7, 2, 3, 4, 8, 0, 2, 6, 8, 9, 7, 1, 9, 3, 7, 2, 7, 9, 6, 9, 5, 7, 8, 5, 9, 8, 3, 9, 8, 0, 5, 5, 7, 1, 2, 0, 7, 5, 8, 2, 2, 5, 3, 9, 3, 1, 9, 3, 1, 4, 3, 4, 4, 9, 0, 9, 5, 9, 2, 9, 4, 0, 2, 8, 4, 8, 6, 8, 2, 0, 5, 9, 6, 9, 5, 1, 3, 4, 4, 2, 7, 2, 4, 4, 0, 2, 8, 4, 5, 1, 2, 6, 8, 1, 7, 2, 8, 7, 4, 3, 3, 0, 3, 7, 2, 5, 6, 1, 7, 9, 0, 2, 3, 9, 8, 9, 5, 0, 0, 7, 6, 3, 3, 8, 1, 4, 0, 1, 5, 4, 3, 2, 6, 0, 8, 8, 6, 3, 3, 2, 9, 7, 5, 1, 6, 0, 5, 9, 9, 4, 5, 9, 4, 8, 3, 3, 2, 5, 1, 9, 5, 5, 8, 1, 8, 9, 7, 0, 6, 3, 2, 0, 2, 6, 9, 3, 9, 1, 6, 6, 7, 2, 6, 3, 2, 5, 5, 2, 7, 5, 2, 0, 8, 0, 7, 7, 1, 7, 4, 0, 2, 2, 6, 1, 5, 9, 7, 6, 2, 7, 0, 5, 6, 0, 1, 1, 8, 4, 5, 3, 1, 2, 4, 8, 9, 8, 1, 7, 2, 7, 2, 3, 2, 6, 7, 9, 4, 0, 1, 0, 4, 5, 0, 8, 0, 7, 6, 1, 0, 8, 5, 9, 2, 5, 4, 4, 9, 0, 6, 1, 2, 5, 1, 6, 7, 1, 5, 8, 8, 0, 3, 9, 4, 0, 3, 4, 9, 2, 4, 3, 0, 6, 4, 5, 6, 6, 7, 8, 4, 8, 8, 3, 2, 8, 0, 2, 5, 6, 8, 7, 8, 9, 9, 0, 6, 6, 8, 1, 1, 7, 4, 4, 2, 3, 6, 9, 6, 2, 4, 1, 8, 2, 3, 1, 6, 8, 6, 8, 5, 9, 0, 1, 0, 2, 9, 2, 9, 0, 9, 7, 8, 5, 0, 6, 6, 8, 1, 6, 5, 8, 7, 9, 2, 1, 6, 7, 5, 6, 3, 0, 9, 8, 9, 5, 9, 9, 9, 3, 6, 4, 4, 8, 8, 1, 7, 3, 0, 6, 2, 4, 8, 2, 2, 9, 2, 2, 9, 7, 4, 1, 9, 1, 4, 3, 5, 3, 8, 9, 3, 6, 2, 0, 8, 9, 0, 5, 1, 5, 8, 2, 1, 2, 3, 2, 4, 8, 4, 0, 6, 4, 2, 9, 8, 4, 7, 6, 2, 4, 7, 7, 6, 4, 2, 2, 3, 2, 4, 9, 0, 0, 9, 6, 5, 8, 5, 2, 4, 8, 8, 6, 4, 5, 7, 6, 3, 7, 0, 4, 0, 5, 8, 2, 8, 0, 4, 9, 2, 3, 6, 8, 1, 6, 0, 8, 4, 3, 1, 4, 5, 2, 4, 3, 6, 0, 7, 1, 3, 5, 7, 1, 8, 0, 4, 8, 2, 7, 7, 0, 2, 2, 4, 3, 3, 1, 6, 1, 6, 4, 6, 7, 4, 7, 3, 8, 1, 3, 1, 4, 6, 5, 5, 7, 4, 5, 4, 9, 7, 7, 2, 3, 3, 6, 7, 4, 1, 6, 1, 0, 6, 9, 6, 0, 2, 0, 2, 4, 7, 0, 0, 0, 0, 7, 5, 3, 2, 7, 8, 5, 5, 2, 2, 8, 9, 1, 2, 2, 0, 3, 4, 9, 1, 6, 9, 0, 8, 3, 6, 4, 6, 7, 5, 1, 8, 9, 0, 5, 0, 5, 4, 3, 8, 5, 2, 0, 8, 5, 9, 2, 5, 6, 5, 7, 3, 7, 2, 8, 8, 4, 2, 1, 6, 2, 9, 9, 8, 2, 3, 1, 0, 5, 2, 3, 7, 9, 5, 9, 1, 1, 3, 2, 7, 4, 4, 1, 9, 4, 5, 5, 2, 7, 7, 6, 1, 2, 3, 7, 5, 4, 3, 7, 7, 0, 0, 1, 4, 4, 3, 7, 9, 8, 7, 0, 9, 0, 3, 7, 6, 3, 8, 4, 5, 6, 5, 3, 4, 2, 9, 2, 4, 9, 2, 0, 1, 1, 3, 7, 9, 5, 1, 6, 0, 3, 3, 7, 2, 4, 5, 2, 3, 6, 3, 3, 0, 4, 4, 3, 8, 1, 2, 3, 2, 7, 1, 4, 5, 6, 8, 6, 6, 5, 4, 2, 9, 9, 3, 2, 0, 4, 1, 5, 5, 6, 6, 5, 6, 9, 7, 9, 4, 8, 3, 1, 9, 8, 1, 9, 0, 3, 0, 4, 7, 7, 2, 6, 5, 6, 9, 7, 7, 5, 7, 0, 8, 6, 7, 8, 3, 8, 9, 3, 2, 9, 2, 7, 4, 1, 9, 7, 0, 8, 1, 2, 0, 6, 5, 1, 4, 2, 8, 2, 8, 3, 3, 1, 4, 9, 8, 1, 1, 7, 9, 9, 4, 6, 8, 5, 3, 2, 2, 3, 8, 5, 1, 9, 7, 9, 1, 5, 3, 1, 0, 3, 8, 9, 2, 0, 8, 0, 7, 9, 4, 8, 3, 0, 7, 0, 7, 0, 5, 5, 4, 3, 0, 9, 9, 0, 2, 2, 0, 8, 4, 4, 5, 2, 2, 1, 1, 5, 5, 4, 3, 4, 0, 8, 4, 8, 2, 5, 6, 5, 1, 7, 2, 4, 0, 5, 6, 8, 7, 6, 3, 1, 3, 6, 6, 4, 5, 5, 5, 1, 8, 7, 3, 0, 2, 5, 1, 1, 1, 6, 7, 2, 1, 2, 3, 9, 8, 3, 1, 7, 4, 8, 4, 7, 5, 8, 6, 0, 3, 0, 8, 3, 4, 4, 9, 4, 6, 1, 5, 7, 7, 9, 0, 1, 3, 1, 3, 2, 4, 0, 7, 8, 3, 1, 6, 2, 1, 8, 8, 1, 9, 7, 5, 3, 2, 9, 0, 6, 8, 4, 1, 5, 7, 2, 4, 7, 0, 4, 9, 0, 7, 0, 6, 7, 1, 8, 5, 8, 8, 7, 0, 0, 3, 2, 5, 7, 0, 6, 4, 4, 9, 1, 8, 6, 3, 9, 4, 9, 3, 3, 7, 5, 9, 7, 7, 4, 5, 8, 2, 8, 9, 5, 3, 1, 9, 2, 2, 0, 1, 1, 8, 1, 1, 9, 8, 9, 9, 0, 2, 6, 9, 1, 9, 0, 1, 9, 2, 9, 9, 9, 0, 9, 9, 2, 1, 7, 0, 5, 6, 3, 9, 8, 3, 0, 2, 7, 7, 4, 8, 3, 0, 5, 2, 3, 5, 7, 6, 6, 6, 5, 6, 5, 4, 5, 8, 8, 0, 9, 7, 9, 0, 6, 9, 6, 1, 4, 7, 9, 0, 3, 5, 4, 7, 6, 9, 8, 2, 3, 9, 3, 3, 7, 8, 3, 5, 5, 9, 6, 1, 4, 7, 4, 8, 7, 7, 1, 3, 0, 3, 2, 5, 4, 9, 3, 5, 4, 7, 0, 3, 7, 0, 2, 1, 5, 8, 7, 3, 5, 7, 8, 5, 7, 8, 1, 5, 4, 7, 0, 8, 3, 2, 9, 7, 4, 1, 6, 5, 9, 8, 5, 5, 4, 1, 0, 9, 4, 4, 3, 0, 4, 8, 0, 8, 2, 9, 5, 9, 7, 4, 6, 7, 9, 2, 9, 3, 7, 7, 8, 2, 2, 0, 2, 5, 3, 6, 4, 7, 2, 3, 7, 8, 7, 2, 5, 0, 0, 7, 0, 9, 6, 1, 0, 3, 9, 7, 4, 9, 1, 6, 8, 1, 2, 3, 3, 5, 4, 8, 9, 7, 4, 4, 1, 2, 4, 9, 8, 7, 9, 5, 1, 2, 1, 6, 6, 4, 5, 7, 4, 5, 8, 5, 2, 8, 7, 8, 2, 3, 6, 1, 3, 3, 1, 5, 1, 9, 0, 9, 2, 0, 6, 2, 4, 8, 5, 7, 6, 1, 2, 9, 4, 5, 0, 3, 3, 7, 7, 7, 1, 4, 5, 0, 2, 8, 5, 0, 0, 6, 2, 0, 8, 4, 5, 4, 5, 6, 4, 7, 9, 4, 2, 0, 6, 4, 0, 0, 6, 4, 6, 1, 9, 5, 5, 2, 2, 6, 3, 4, 5, 9, 1, 7, 2, 3, 9, 6, 5, 0, 2, 9, 7, 1, 7, 2, 2, 0, 8, 6, 4, 3, 2, 7, 7, 0, 4, 1, 6, 5, 1, 3, 0, 3, 9, 0, 0, 2, 5, 0, 4, 0, 1, 9, 8, 4, 9, 4, 2, 4, 3, 3, 4, 0, 4, 3, 2, 8, 9, 1, 5, 8, 1, 8, 2, 4, 5, 2, 4, 1, 1, 6, 6, 8, 5, 2, 2, 5, 0, 8, 2, 3, 6, 2, 9, 6, 1, 4, 5, 9, 0, 1, 0, 0, 8, 1, 1, 6, 6, 9, 5, 4, 1, 7, 8, 6, 9, 1, 7, 6, 0, 9, 3, 5, 3, 2, 5, 3, 4, 9, 7, 1, 4, 4, 6, 1, 3, 8, 8, 0, 6, 7, 7, 6, 7, 2, 3, 2, 2, 6, 2, 7, 4, 0, 3, 6, 2, 6, 3, 3, 0, 9, 5, 1, 1, 5, 3, 6, 4, 3, 4, 1, 0, 4, 5, 5, 2, 8, 9, 4, 3, 1, 8, 0, 1, 3, 3, 4, 4, 2, 9, 7, 6, 8, 1, 8, 9, 1, 3, 1, 7, 3, 0, 0, 2, 8, 3, 9, 2, 7, 2, 6, 0, 1, 6, 1, 6, 7, 5, 5, 2, 5, 9, 4, 0, 2, 3, 4, 9, 4, 1, 0, 0, 2, 3, 8, 9, 2, 8, 9, 5, 7, 9, 1, 4, 6, 2, 8, 4, 4, 8, 9, 3, 1, 1, 6, 5, 8, 4, 6, 4, 5, 2, 6, 4, 2, 3, 1, 3, 6, 8, 5, 2, 2, 7, 1, 6, 2, 5, 9, 2, 8, 1, 6, 9, 2, 7, 5, 3, 2, 9, 7, 0, 2, 9, 3, 4, 1, 7, 9, 5, 8, 9, 7, 3, 4, 0, 9, 7, 4, 2, 4, 7, 0, 1, 8, 1, 0, 4, 6, 1, 9, 9, 2, 1, 2, 5, 6, 9, 7, 7, 3, 4, 2, 0, 2, 5, 6, 7, 3, 7, 9, 4, 2, 0, 6, 1, 7, 5, 6, 5, 3, 9, 2, 7, 8, 5, 9, 5, 8, 5, 4, 3, 7, 9, 8, 1, 2, 2, 8, 9, 3, 0, 8, 4, 0, 2, 0, 1, 4, 1, 8, 5, 5, 7, 9, 8, 3, 7, 9, 1, 5, 6, 9, 8, 7, 2, 0, 9, 0, 8, 5, 9, 4, 2, 9, 8, 1, 9, 1, 8, 3, 7, 6, 4, 2, 3, 7, 0, 3, 5, 8, 8, 8, 7, 9, 6, 2, 7, 4, 5, 7, 6, 7, 1, 7, 3, 6, 8, 2, 6, 6, 7, 1, 5, 9, 7, 1, 7, 0, 1, 6, 3, 3, 9, 0, 1, 2, 3, 2, 2, 5, 4, 9, 8, 7, 4, 4, 9, 7, 6, 7, 7, 1, 2, 3, 5, 1, 9, 0, 3, 3, 1, 5, 6, 6, 2, 4, 6, 8, 8, 9, 6, 6, 1, 0, 7, 5, 8, 2, 1, 5, 8, 1, 4, 7, 5, 0, 3, 9, 9, 5, 2, 8, 4, 1, 9, 0, 4, 4, 8, 2, 9, 0, 7, 9, 8, 7, 1, 3, 2, 9, 9, 5, 9, 7, 6, 7, 7, 1, 1, 2, 4, 2, 0, 6, 8, 7, 6, 2, 2, 9, 8, 2, 4, 2, 0, 5, 8, 6, 8, 2, 7, 7, 3, 1, 8, 1, 6, 5, 9, 7, 8, 9, 6, 4, 8, 1, 9, 4, 0, 4, 1, 4, 3, 6, 2, 2, 7, 0, 0, 7, 0, 7, 4, 3, 6, 7, 7, 4, 5, 4, 3, 5, 5, 4, 7, 1, 0, 7, 7, 1, 6, 5, 0, 7, 7, 4, 6, 1, 8, 5, 9, 3, 5, 6, 2, 2, 7, 3, 5, 8, 1, 0, 6, 8, 7, 8, 8, 5, 7, 5, 4, 9, 7, 3, 3, 8, 8, 3, 9, 4, 7, 2, 0, 8, 0, 7, 3, 3, 2, 5, 2, 4, 4, 0, 4, 8, 2, 4, 0, 6, 4, 5, 6, 0, 8, 8, 0, 6, 1, 1, 6, 1, 4, 2, 1, 1, 2, 4, 4, 5, 5, 8, 8, 5, 1, 8, 2, 3, 3, 9, 6, 6, 5, 0, 7, 3, 3, 2, 7, 4, 5, 6, 0, 2, 1, 8, 1, 0, 9, 3, 1, 0, 5, 4, 2, 3, 3, 6, 7, 6, 0, 0, 5, 7, 4, 7, 7, 0, 6, 1, 3, 9, 0, 9, 0, 3, 8, 4, 8, 8, 4, 1, 0, 2, 2, 4, 1, 1, 2, 4, 3, 4, 1, 4, 7, 1, 0, 0, 9, 0, 4, 2, 8, 8, 5, 5, 0, 4, 0, 8, 6, 2, 5, 9, 2, 9, 1, 1, 5, 4, 7, 5, 8, 6, 2, 1, 5, 5, 3, 4, 1, 8, 9, 9, 8, 9, 8, 6, 8, 5, 8, 9, 4, 6, 2, 6, 3, 7, 4, 0, 0, 1, 7, 5, 1, 5, 9, 3, 1, 6, 8, 7, 3, 6, 9, 1, 2, 0, 1, 7, 2, 6, 1, 9, 0, 0, 8, 9, 9, 2, 8, 6, 2, 5, 6, 0, 3, 3, 0, 7, 4, 7, 5, 0, 1, 6, 8, 8, 1, 2, 1, 5, 4, 5, 9, 6, 7, 1, 0, 6, 9, 2, 7, 7, 3, 9, 9, 1, 9, 7, 0, 1, 3, 5, 4, 6, 3, 8, 8, 0, 4, 8, 3, 6, 7, 0, 0, 4, 5, 2, 6, 8, 4, 9, 9, 9, 2, 0, 4, 2, 8, 1, 1, 0, 0, 3, 7, 4, 1, 1, 9, 7, 7, 4, 6, 8, 6, 0, 2, 8, 5, 3, 5, 3, 5, 7, 9, 8, 4, 4, 3, 1, 4, 8, 3, 6, 5, 3, 0, 8, 9, 5, 7, 6, 2, 0, 4, 9, 9, 0, 5, 2, 3, 6, 1, 1, 0, 2, 8, 2, 1, 1, 7, 5, 2, 3, 4, 1, 2, 9, 2, 1, 3, 4, 8, 9, 0, 0, 4, 9, 0, 2, 2, 0, 6, 8, 7, 3, 3, 8, 9, 0, 2, 5, 3, 6, 1, 3, 9, 5, 0, 5, 4, 4, 0, 1, 1, 6, 1, 7, 9, 9, 6, 2, 4, 3, 8, 3, 4, 7, 0, 2, 8, 4, 8, 3, 8, 8, 8, 3, 3, 5, 7, 7, 0, 4, 1, 5, 9, 7, 0, 6, 8, 4, 9, 0, 1, 8, 9, 6, 3, 9, 2, 4, 4, 0, 3, 3, 5, 4, 5, 1, 1, 8, 2, 2, 9, 3, 7, 8, 9, 2, 3, 1, 7, 3, 2, 3, 0, 1, 9, 5, 5, 3, 4, 5, 2, 0, 0, 3, 1, 3, 4, 7, 4, 2, 8, 4, 8, 9, 9, 4, 2, 4, 3, 6, 4, 6, 4, 6, 6, 3, 8, 7, 4, 8, 0, 5, 9, 3, 9, 0, 7, 5, 1, 9, 7, 9, 1, 8, 4, 2, 1, 6, 4, 3, 0, 1, 4, 8, 6, 1, 7, 1, 3, 5, 4, 3, 3, 9, 7, 8, 2, 5, 5, 4, 5, 4, 5, 7, 2, 5, 5, 1, 7, 7, 8, 3, 6, 0, 2, 5, 2, 3, 7, 4, 2, 1, 6, 8, 6, 8, 3, 2, 7, 7, 9, 7, 1, 4, 7, 4, 6, 1, 7, 3, 0, 0, 6, 6, 8, 6, 6, 0, 3, 4, 7, 4, 9, 4, 9, 9, 3, 4, 1, 4, 0, 3, 7, 1, 2, 2, 8, 4, 7, 8, 5, 5, 6, 5, 6, 0, 6, 4, 9, 3, 2, 7, 3, 8, 3, 9, 4, 1, 6, 9, 9, 4, 3, 9, 8, 1, 6, 9, 5, 9, 0, 9, 7, 2, 2, 4, 2, 6, 4, 8, 1, 9, 5, 6, 5, 1, 8, 6, 6, 7, 6, 5, 0, 2, 7, 6, 1, 2, 3, 1, 6, 4, 9, 0, 3, 9, 1, 0, 5, 3, 1, 6, 9, 8, 9, 0, 6, 1, 6, 2, 3, 5, 6, 9, 0, 7, 2, 4, 0, 3, 6, 8, 6, 9, 1, 9, 0, 6, 4, 5, 9, 5, 4, 1, 9, 2, 0, 7, 3, 5, 1, 8, 3, 0, 5, 2, 8, 8, 3, 9, 6, 0, 3, 5, 3, 4, 1, 5, 0, 7, 3, 9, 4, 5, 3, 1, 4, 2, 4, 9, 9, 7, 1, 7, 0, 1, 2, 1, 5, 3, 8, 4, 1, 5, 1, 9, 9, 7, 0, 1, 7, 6, 2, 6, 5, 0, 3, 1, 3, 8, 9, 9, 0, 8, 8, 7, 9, 2, 9, 1, 3, 6, 7, 6, 0, 8, 6, 2, 4, 2, 3, 5, 5, 4, 1, 6, 7, 1, 6, 1, 7, 1, 6, 2, 3, 0, 4, 2, 9, 7, 5, 5, 6, 0, 8, 8, 1, 1, 3, 5, 6, 5, 3, 8, 7, 5, 0, 7, 7, 5, 7, 1, 9, 9, 1, 9, 9, 3, 5, 5, 3, 0, 3, 0, 6, 6, 5, 6, 8, 9, 4, 5, 1, 1, 2, 7, 9, 0, 1, 2, 5, 4, 4, 7, 4, 8, 3, 3, 5, 3, 5, 7, 0, 1, 2, 9, 8, 1, 7, 3, 5, 9, 1, 9, 1, 1, 3, 6, 7, 4, 8, 3, 5, 6, 7, 0, 1, 7, 7, 2, 3, 3, 5, 1, 7, 6, 8, 4, 4, 5, 7, 3, 4, 7, 7, 3, 6, 0, 7, 2, 6, 7, 4, 1, 8, 0, 2, 2, 1, 5, 4, 0, 9, 3, 4, 4, 2, 3, 8, 5, 0, 0, 4, 6, 3, 9, 2, 0, 5, 7, 5, 5, 1, 7, 3, 1, 6, 7, 9, 0, 1, 2, 1, 4, 6, 1, 5, 9, 5, 0, 4, 9, 4, 9, 7, 1, 6, 8, 0, 6, 5, 0, 8, 1, 7, 5, 6, 9, 3, 2, 3, 9, 8, 6, 1, 0, 9, 0, 9, 8, 7, 7, 5, 2, 3, 5, 5, 7, 8, 7, 6, 9, 1, 9, 8, 6, 5, 6, 7, 5, 6, 7, 2, 5, 9, 6, 6, 7, 0, 1, 0, 1, 3, 2, 5, 6, 1, 2, 5, 1, 9, 9, 8, 6, 8, 6, 8, 9, 0, 4, 0, 3, 3, 6, 4, 9, 9, 7, 2, 3, 5, 6, 4, 6, 0, 3, 7, 5, 0, 2, 9, 2, 7, 2, 9, 2, 4, 0, 6, 6, 3, 4, 5, 5, 9, 1, 1, 4, 4, 3, 5, 4, 8, 0, 0, 3, 9, 4, 7, 4, 2, 8, 3, 6, 3, 1, 9, 6, 3, 8, 3, 8, 4, 7, 5, 7, 5, 3, 1, 2, 6, 1, 3, 2, 0, 7, 5, 0, 0, 0, 1, 4, 3, 5, 8, 4, 3, 1, 7, 1, 0, 4, 2, 4, 1, 4, 0, 1, 2, 1, 7, 7, 9, 8, 7, 5, 4, 0, 9, 0, 0, 8, 2, 0, 0, 2, 4, 8, 6, 2, 4, 6, 3, 5, 1, 5, 3, 7, 2, 2, 9, 8, 0, 0, 0, 3, 4, 4, 6, 1, 6, 7, 4, 4, 3, 9, 4, 0, 8, 0, 4, 6, 5, 7, 9, 7, 0, 5, 7, 7, 3, 1, 9, 3, 0, 9, 5, 3, 7, 9, 4, 4, 1, 7, 7, 1, 4, 1, 2, 8, 7, 0, 0, 4, 7, 2, 9, 7, 6, 9, 3, 5, 8, 0, 3, 6, 8, 3, 2, 4, 7, 1, 1, 3, 9, 7, 5, 1, 0, 8, 7, 0, 1, 6, 9, 3, 2, 7, 7, 8, 1, 0, 3, 4, 6, 7, 5, 2, 0, 1, 5, 5, 1, 4, 1, 3, 0, 8, 6, 2, 1, 3, 6, 4, 1, 9, 0, 4, 1, 0, 1, 9, 8, 6, 9, 2, 4, 7, 2, 2, 7, 4, 9, 1, 3, 2, 6, 3, 4, 4, 9, 4, 8, 2, 6, 6, 1, 6, 3, 6, 5, 8, 4, 6, 7, 1, 9, 3, 6, 7, 6, 0, 7, 1, 9, 5, 2, 6, 7, 7, 6, 5, 9, 1, 5, 6, 0, 2, 0, 9, 1, 8, 3, 5, 0, 0, 0, 5, 7, 7, 8, 5, 5, 5, 1, 6, 1, 5, 1, 0, 6, 2, 3, 2, 1, 7, 5, 1, 9, 8, 3, 6, 9, 7, 3, 0, 2, 3, 9, 4, 4, 3, 9, 6, 8, 6, 6, 8, 5, 4, 3, 6, 7, 7, 4, 3, 9, 6, 2, 4, 0, 1, 3, 6, 4, 9, 2, 6, 0, 3, 8, 7, 5, 3, 3, 8, 3, 2, 6, 3, 6, 2, 7, 4, 5, 7, 9, 0, 0, 6, 7, 2, 8, 5, 5, 7, 5, 5, 9, 4, 6, 4, 7, 3, 3, 6, 4, 1, 6, 3, 1, 6, 7, 0, 5, 0, 1, 9, 7, 3, 5, 2, 3, 9, 5, 6, 4, 0, 0, 0, 8, 0, 3, 2, 4, 5, 3, 7, 9, 6, 9, 3, 1, 2, 6, 7, 4, 5, 1, 3, 7, 6, 9, 5, 8, 5, 8, 5, 8, 5, 6, 5, 0, 8, 3, 8, 1, 8, 1, 5, 0, 9, 8, 6, 3, 6, 3, 4, 4, 7, 4, 7, 4, 3, 2, 4, 5, 5, 7, 5, 4, 5, 8, 0, 5, 4, 0, 5, 4, 3, 3, 2, 4, 2, 9, 4, 8, 8, 6, 2, 6, 1, 7, 0, 4, 3, 8, 5, 9, 7, 6, 7, 1, 0, 2, 3, 5, 6, 7, 1, 6, 2, 8, 3, 7, 2, 5, 7, 5, 7, 7, 1, 7, 4, 3, 3, 4, 0, 4, 2, 7, 0, 2, 2, 6, 6, 2, 5, 2, 6, 0, 6, 1, 2, 9, 0, 0, 1, 5, 5, 1, 6, 7, 8, 5, 6, 3, 3, 6, 0, 9, 9, 2, 2, 6, 1, 4, 6, 2, 5, 7, 8, 8, 5, 6, 4, 3, 2, 3, 4, 4, 4, 8, 9, 2, 6, 0, 9, 7, 9, 8, 7, 7, 3, 8, 2, 9, 3, 5, 7, 5, 8, 7, 3, 7, 0, 7, 1, 6, 2, 4, 0, 0, 2, 9, 8, 2, 8, 7, 7, 5, 5, 0, 1, 2, 9, 6, 1, 0, 1, 0, 6, 5, 0, 5, 7, 4, 4, 8, 3, 5, 3, 7, 2, 5, 8, 3, 5, 7, 3, 7, 0, 5, 4, 5, 6, 9, 3, 5, 3, 8, 2, 4, 2, 2, 6, 7, 6, 0, 5, 5, 8, 4, 5, 6, 1, 3, 5, 4, 7, 3, 0, 7, 2, 0, 1, 2, 4, 9, 6, 4, 5, 9, 7, 7, 6, 7, 4, 3, 1, 6, 9, 4, 8, 0, 3, 1, 6, 4, 4, 2, 1, 4, 6, 3, 0, 8, 9, 7, 6, 8, 4, 8, 1, 4, 5, 9, 4, 7, 1, 4, 5, 3, 1, 2, 6, 4, 7, 1, 5, 9, 3, 1, 4, 8, 7, 7, 0, 4, 3, 4, 6, 7, 7, 5, 4, 4, 4, 3, 6, 6, 1, 9, 9, 7, 9, 4, 3, 2, 9, 8, 5, 0, 6, 9, 0, 9, 7, 8, 5, 4, 4, 9, 4, 2, 6, 6, 9, 9, 7, 9, 9, 0, 5, 1, 1, 8, 6, 9, 9, 5, 6, 5, 5, 9, 7, 9, 4, 5, 4, 3, 3, 9, 8, 8, 3, 4, 8, 4, 0, 4, 0, 7, 2, 8, 0, 2, 0, 7, 4, 0, 6, 3, 3, 1, 7, 4, 6, 5, 0, 4, 9, 5, 6, 6, 3, 0, 5, 2, 6, 3, 3, 8, 0, 8, 5, 4, 2, 9, 5, 0, 5, 4, 8, 2, 7, 6, 5, 7, 8, 2, 0, 1, 8, 2, 4, 8, 4, 3, 0, 4, 0, 1, 9, 3, 0, 3, 6, 5, 5, 2, 8, 5, 0, 8, 7, 5, 2, 4, 3, 7, 0, 2, 3, 1, 0, 0, 2, 1, 2, 1, 4, 6, 7, 1, 0, 5, 2, 5, 9, 2, 5, 6, 7, 1, 9, 3, 6, 1, 0, 2, 9, 4, 2, 9, 3, 8, 2, 9, 8, 5, 1, 0, 1, 5, 2, 5, 6, 8, 8, 3, 5, 0, 4, 1, 3, 7, 0, 3, 3, 9, 2, 3, 9, 9, 8, 5, 9, 7, 2, 0, 9, 4, 7, 1, 7, 6, 4, 4, 8, 9, 0, 7, 1, 2, 4, 0, 3, 7, 5, 1, 9, 3, 5, 2, 4, 1, 1, 3, 3, 9, 2, 5, 8, 0, 8, 1, 5, 1, 5, 9, 0, 9, 2, 4, 0, 6, 9, 1, 1, 3, 4, 9, 5, 3, 3, 3, 3, 4, 1, 5, 9, 8, 7, 4, 8, 7, 3, 5, 2, 5, 8, 9, 7, 2, 0, 0, 7, 5, 3, 5, 7, 4, 6, 2, 8, 2, 7, 7, 6, 1, 3, 8, 6, 2, 0, 4, 4, 8, 6, 4, 0, 2, 5, 4, 1, 2, 5, 4, 6, 9, 5, 0, 5, 2, 0, 8, 2, 6, 4, 7, 9, 5, 7, 2, 6, 2, 1, 9, 9, 7, 2, 5, 1, 8, 1, 6, 3, 0, 8, 8, 5, 6, 9, 9, 9, 6, 3, 0, 8, 8, 5, 3, 3, 1, 1, 0, 7, 0, 6, 0, 4, 4, 6, 5, 3, 2, 9, 6, 6, 7, 9, 7, 7, 0, 8, 4, 2, 7, 7, 7, 4, 0, 1, 6, 0, 5, 3, 0, 2, 4, 9, 7, 3, 3, 2, 8, 8, 5, 3, 8, 0, 8, 0, 0, 0, 2, 2, 7, 0, 7, 7, 7, 2, 2, 0, 4, 0, 3, 7, 8, 4, 7, 9, 6, 1, 1, 6, 9, 3, 8, 9, 5, 8, 3, 9, 4, 3, 7, 5, 5, 2, 7, 5, 7, 9, 6, 3, 0, 8, 4, 5, 6, 1, 0, 4, 8, 4, 7, 1, 3, 2, 0, 9, 1, 8, 8, 6, 8, 1, 7, 1, 0, 9, 7, 6, 4, 5, 8, 2, 1, 2, 5, 8, 1, 1, 8, 8, 0, 9, 0, 0, 4, 3, 3, 5, 6, 6, 3, 1, 4, 4, 7, 8, 0, 1, 4, 8, 4, 6, 2, 2, 2, 6, 3, 4, 0, 8, 1, 3, 8, 3, 8, 9, 1, 6, 1, 9, 8, 2, 0, 4, 7, 4, 0, 0, 7, 6, 8, 9, 8, 6, 2, 8, 8, 0, 1, 1, 0, 7, 7, 4, 4, 7, 7, 2, 6, 6, 7, 4, 5, 4, 0, 9, 4, 7, 2, 5, 3, 2, 2, 7, 6, 2, 6, 4, 9, 6, 8, 3, 8, 5, 5, 2, 3, 4, 5, 2, 7, 5, 1, 5, 3, 7, 8, 8, 9, 6, 0, 9, 2, 6, 0, 0, 1, 4, 1, 3, 5, 1, 8, 9, 6, 2, 6, 9, 5, 6, 5, 9, 1, 4, 2, 8, 9, 5, 4, 7, 0, 3, 7, 1, 6, 9, 1, 1, 4, 0, 5, 1, 6, 3, 0, 2, 5, 5, 5, 2, 8, 2, 3, 1, 6, 2, 7, 0, 8, 9, 2, 6, 4, 7, 5, 5, 8, 8, 7, 8, 5, 7, 4, 4, 0, 0, 7, 0, 4, 0, 4, 4, 2, 5, 4, 0, 7, 7, 7, 0, 4, 6, 2, 9, 1, 1, 5, 0, 9, 1, 0, 6, 5, 3, 0, 9, 0, 6, 2, 0, 4, 1, 6, 7, 6, 1, 0, 4, 6, 0, 4, 4, 2, 7, 6, 5, 3, 5, 9, 4, 5, 2, 0, 5, 7, 2, 3, 9, 4, 5, 8, 1, 9, 3, 1, 8, 5, 8, 6, 6, 3, 4, 0, 2, 3, 2, 6, 1, 8, 2, 1, 3, 6, 2, 2, 3, 3, 5, 0, 2, 1, 2, 8, 0, 8, 2, 2, 7, 3, 5, 9, 8, 3, 6, 1, 0, 8, 6, 3, 5, 8, 9, 4, 8, 0, 0, 2, 9, 1, 8, 3, 7, 2, 8, 0, 9, 4, 8, 9, 4, 8, 9, 5, 4, 8, 1, 5, 5, 7, 2, 5, 8, 8, 1, 2, 3, 1, 5, 0, 2, 3, 0, 8, 2, 2, 8, 7, 3, 5, 3, 9, 6, 3, 5, 1, 1, 7, 7, 3, 3, 8, 9, 6, 0, 8, 0, 6, 6, 4, 6, 9, 1, 1, 7, 0, 6, 8, 9, 0, 6, 9, 0, 1, 1, 7, 2, 1, 0, 4, 5, 0, 3, 6, 3, 3, 8, 1, 8, 1, 3, 1, 5, 4, 5, 1, 8, 9, 1, 3, 1, 7, 2, 9, 8, 9, 4, 6, 0, 9, 4, 0, 4, 6, 2, 1, 3, 0, 8, 5, 6, 0, 3, 4, 6, 9, 5, 7, 9, 0, 3, 6, 5, 7, 0, 5, 1, 2, 2, 9, 7, 0, 0, 1, 4, 4, 5, 5, 9, 6, 3, 6, 2, 6, 9, 3, 2, 3, 2, 4, 9, 5, 2, 1, 6, 6, 4, 7, 6, 3, 3, 8, 1, 4, 6, 4, 4, 5, 6, 2, 0, 5, 2, 7, 5, 2, 5, 2, 5, 9, 0, 6, 7, 9, 8, 3, 2, 1, 4, 3, 4, 5, 5, 6, 8, 6, 0, 8, 9, 8, 0, 7, 7, 6, 6, 7, 5, 9, 8, 7, 9, 5, 6, 0, 4, 8, 2, 0, 9, 1, 3, 9, 8, 3, 7, 4, 5, 0, 2, 1, 9, 1, 0, 5, 8, 4, 7, 3, 1, 4, 5, 3, 1, 0, 8, 8, 8, 2, 1, 3, 7, 3, 1, 6, 9, 1, 6, 0, 7, 7, 1, 3, 3, 3, 1, 0, 2, 7, 1, 2, 2, 9, 6, 3, 5, 9, 4, 8, 0, 4, 8, 0, 1, 3, 7, 0, 4, 7, 4, 8, 6, 4, 3, 9, 8, 2, 2, 8, 3, 1, 1, 2, 8, 2, 6, 9, 4, 8, 4, 4, 1, 5, 2, 6, 9, 2, 0, 7, 1, 8, 9, 3, 9, 9, 0, 7, 7, 5, 4, 2, 6, 4, 5, 7, 7, 8, 7, 2, 6, 2, 2, 4, 4, 0, 7, 1, 3, 9, 6, 0, 0, 2, 3, 8, 2, 2, 4, 3, 5, 2, 9, 1, 0, 0, 6, 5, 5, 7, 9, 9, 6, 5, 5, 0, 5, 7, 1, 6, 6, 4, 1, 4, 4, 1, 5, 0, 0, 4, 5, 8, 4, 8, 3, 0, 5, 0, 5, 3, 1, 6, 7, 0, 9, 1, 5, 7, 6, 5, 5, 5, 6, 0, 0, 1, 7, 5, 1, 9, 2, 4, 1, 3, 7, 8, 2, 0, 9, 6, 6, 0, 6, 5, 8, 2, 7, 4, 0, 2, 7, 7, 8, 8, 7, 0, 4, 9, 1, 4, 4, 3, 5, 4, 6, 2, 3, 1, 0, 3, 3, 3, 6, 3, 1, 2, 8, 9, 7, 9, 3, 8, 7, 3, 1, 7, 7, 3, 2, 2, 8, 9, 5, 9, 2, 1, 7, 4, 4, 0, 5, 7, 1, 5, 4, 0, 8, 4, 9, 8, 7, 8, 4, 2, 3, 4, 0, 5, 4, 1, 8, 2, 5, 4, 5, 2, 5, 3, 7, 9, 7, 1, 4, 1, 3, 1, 4, 5, 5, 1, 7, 1, 3, 0, 1, 2, 5, 7, 0, 6, 3, 5, 7, 5, 5, 8, 9, 4, 6, 3, 6, 6, 8, 2, 6, 4, 8, 4, 1, 3, 1, 2, 3, 3, 0, 1, 6, 4, 2, 8, 2, 4, 3, 1, 4, 4, 4, 4, 1, 5, 5, 7, 6, 5, 4, 5, 6, 5, 6, 2, 4, 7, 7, 4, 6, 5, 0, 2, 3, 9, 2, 3, 8, 0, 7, 0, 6, 8, 1, 2, 8, 9, 1, 5, 4, 2, 3, 5, 5, 3, 6, 5, 0, 3, 2, 1, 3, 3, 7, 7, 4, 9, 4, 3, 9, 5, 1, 7, 6, 6, 4, 8, 5, 1, 2, 8, 4, 5, 3, 3, 3, 0, 4, 8, 7, 7, 9, 4, 8, 4, 5, 9, 1, 8, 3, 1, 3, 8, 0, 3, 5, 4, 6, 9, 2, 7, 2, 0, 5, 2, 9, 7, 5, 0, 7, 0, 3, 5, 5, 3, 8, 7, 7, 9, 6, 5, 8, 9, 0, 1, 9, 4, 0, 0, 1, 7, 2, 2, 8, 5, 9, 8, 2, 8, 0, 6, 6, 8, 6, 9, 6, 1, 1, 5, 3, 5, 0, 7, 6, 1, 4, 4, 1, 1, 8, 1, 1, 2, 3, 4, 9, 9, 5, 2, 7, 3, 3, 6, 9, 6, 9, 8, 8, 6, 7, 4, 7, 2, 3, 8, 0, 9, 0, 7, 8, 1, 2, 3, 2, 4, 2, 9, 0, 3, 8, 9, 6, 5, 5, 2, 4, 1, 0, 6, 1, 8, 4, 5, 5, 5, 0, 8, 9, 0, 5, 7, 9, 6, 3, 4, 6, 4, 9, 7, 1, 2, 5, 2, 8, 0, 2, 2, 9, 8, 1, 9, 7, 7, 3, 1, 4, 1, 1, 3, 8, 8, 1, 9, 3, 8, 9, 5, 1, 1, 6, 2, 7, 2, 6, 2, 6, 9, 6, 0, 1, 3, 9, 3, 8, 8, 8, 2, 9, 1, 2, 6, 0, 5, 0, 7, 7, 7, 9, 4, 5, 0, 2, 3, 5, 5, 8, 9, 1, 2, 7, 1, 3, 4, 6, 9, 1, 0, 7, 1, 9, 5, 8, 1, 9, 9, 7, 9, 8, 3, 0, 2, 6, 0, 2, 2, 4, 0, 8, 4, 8, 1, 2, 7, 2, 7, 9, 5, 2, 2, 9, 2, 6, 5, 7, 2, 5, 6, 5, 1, 8, 2, 6, 8, 6, 3, 0, 8, 0, 0, 9, 9, 6, 8, 0, 6, 6, 0, 7, 9, 8, 0, 5, 0, 9, 6, 0, 4, 7, 1, 7, 8, 8, 2, 3, 9, 6, 5, 5, 3, 1, 5, 3, 0, 2, 5, 2, 7, 3, 8, 4, 4, 6, 1, 9, 4, 1, 7, 0, 1, 4, 3, 0, 2, 9, 4, 5, 0, 3, 4, 7, 2, 3, 0, 1, 3, 7, 9, 9, 0, 7, 5, 6, 7, 1, 2, 2, 7, 4, 6, 8, 4, 9, 8, 4, 9, 2, 3, 4, 0, 2, 5, 6, 0, 5, 3, 5, 8, 3, 7, 4, 9, 5, 6, 3, 8, 6, 9, 8, 5, 5, 8, 3, 5, 9, 0, 4, 2, 0, 5, 9, 9, 3, 8, 0, 6, 0, 2, 3, 7, 0, 0, 6, 9, 1, 5, 2, 2, 7, 9, 3, 9, 3, 5, 2, 2, 2, 8, 8, 5, 0, 0, 7, 0, 3, 1, 6, 5, 0, 9, 2, 6, 7, 4, 2, 0, 0, 5, 3, 3, 2, 6, 4, 2, 7, 8, 0, 1, 0, 1, 3, 6, 4, 8, 0, 6, 0, 9, 1, 6, 2, 1, 0, 1, 3, 0, 3, 7, 9, 9, 3, 7, 5, 7, 1, 6, 6, 1, 1, 1, 4, 4, 2, 1, 7, 0, 1, 2, 1, 1, 3, 2, 6, 9, 1, 3, 2, 0, 8, 7, 3, 4, 6, 4, 6, 9, 4, 8, 9, 3, 0, 5, 7, 2, 0, 2, 9, 9, 3, 8, 7, 1, 1, 2, 2, 2, 5, 9, 1, 2, 6, 1, 2, 1, 8, 9, 2, 3, 1, 9, 9, 4, 4, 5, 5, 0, 3, 3, 7, 1, 7, 7, 9, 8, 6, 3, 5, 5, 1, 6, 2, 7, 8, 4, 3, 3, 5, 9, 6, 8, 5, 1, 0, 6, 7, 0, 9, 6, 2, 0, 8, 7, 1, 2, 6, 9, 1, 6, 4, 7, 1, 9, 0, 0, 5, 2, 8, 3, 8, 2, 8, 4, 0, 8, 3, 6, 1, 2, 3, 0, 5, 0, 4, 1, 7, 2, 0, 1, 3, 9, 1, 6, 1, 6, 1, 2, 8, 7, 0, 9, 5, 8, 0, 0, 1, 7, 9, 7, 1, 9, 2, 1, 9, 4, 3, 6, 0, 2, 0, 0, 9, 9, 8, 9, 4, 5, 4, 3, 0, 7, 8, 7, 3, 7, 5, 4, 6, 5, 9, 5, 2, 4, 2, 7, 9, 6, 7, 4, 7, 1, 2, 3, 1, 1, 3, 6, 8, 3, 7, 7, 0, 8, 1, 5, 2, 5, 5, 3, 6, 3, 5, 9, 8, 4, 6, 8, 0, 4, 1, 7, 8, 4, 9, 7, 2, 6, 3, 8, 8, 6, 7, 3, 8, 1, 1, 5, 3, 7, 9, 9, 9, 8, 5, 9, 9, 6, 8, 8, 4, 8, 1, 0, 1, 8, 8, 6, 5, 9, 0, 6, 1, 2, 6, 6, 3, 5, 6, 5, 5, 3, 3, 6, 8, 9, 4, 4, 3, 8, 3, 6, 0, 8, 5, 7, 8, 4, 6, 5, 3, 1, 8, 3, 0, 4, 4, 3, 3, 5, 3, 3, 2, 7, 7, 8, 2, 0, 5, 5, 6, 1, 1, 3, 9, 9, 3, 6, 7, 1, 8, 3, 9, 8, 7, 8, 8, 0, 3, 2, 4, 8, 7, 5, 7, 1, 8, 7, 3, 5, 4, 4, 3, 8, 6, 1, 1, 6, 3, 2, 2, 8, 9, 8, 3, 3, 8, 1, 1, 0, 5, 2, 0, 5, 0, 5, 8, 6, 3, 9, 6, 6, 3, 9, 4, 8, 3, 4, 3, 1, 1, 2, 9, 7, 8, 5, 5, 8, 6, 9, 0, 8, 3, 5, 1, 3, 3, 5, 5, 3, 8, 1, 4, 6, 4, 2, 4, 3, 8, 9, 6, 6, 6, 5, 5, 2, 7, 7, 4, 4, 9, 3, 8, 3, 7, 7, 8, 5, 3, 9, 1, 6, 7, 3, 3, 3, 8, 5, 6, 1, 3, 7, 9, 6, 1, 6, 4, 9, 0, 5, 7, 4, 9, 1, 0, 0, 2, 7, 8, 3, 2, 7, 6, 1, 6, 6, 4, 4, 2, 1, 8, 4, 4, 7, 8, 5, 8, 9, 4, 3, 2, 2, 9, 3, 0, 1, 1, 1, 1, 6, 8, 7, 1, 3, 0, 4, 5, 0, 3, 0, 6, 2, 0, 9, 8, 6, 8, 5, 8, 2, 9, 6, 0, 6, 1, 8, 7, 3, 2, 2, 6, 5, 6, 9, 2, 5, 4, 9, 8, 5, 3, 9, 3, 5, 7, 1, 8, 5, 8, 6, 2, 5, 1, 3, 0, 4, 1, 9, 5, 4, 9, 2, 1, 4, 2, 5, 9, 8, 7, 2, 1, 6, 9, 7, 3, 6, 5, 8, 0, 2, 2, 4, 9, 0, 5, 5, 4, 2, 3, 1, 0, 3, 0, 4, 4, 9, 1, 3, 3, 0, 2, 1, 1, 0, 4, 1, 9, 4, 8, 1, 1, 1, 5, 9, 8, 1, 6, 7, 2, 9, 9, 7, 8, 8, 1, 9, 7, 8, 9, 1, 6, 6, 3, 3, 1, 3, 0, 8, 8, 3, 6, 6, 7, 1, 0, 7, 6, 4, 8, 2, 0, 6, 6, 8, 6, 3, 1, 1, 8, 6, 7, 0, 5, 4, 3, 2, 1, 7, 8, 2, 0, 1, 7, 4, 8, 7, 6, 6, 7, 0, 6, 4, 9, 2, 3, 2, 1, 3, 8, 6, 8, 1, 9, 6, 7, 6, 5, 0, 6, 0, 1, 3, 1, 1, 5, 3, 4, 3, 3, 8, 1, 6, 6, 1, 8, 8, 6, 2, 7, 4, 6, 8, 9, 3, 3, 5, 5, 3, 1, 4, 1, 9, 5, 1, 7, 7, 7, 5, 2, 3, 3, 6, 6, 6, 4, 0, 7, 1, 7, 3, 0, 8, 1, 2, 4, 1, 2, 2, 4, 9, 8, 2, 8, 7, 6, 3, 4, 7, 2, 3, 1, 5, 3, 0, 4, 3, 4, 9, 4, 7, 9, 1, 7, 8, 3, 1, 8, 3, 2, 5, 7, 2, 7, 9, 6, 8, 6, 8, 6, 5, 9, 0, 4, 8, 5, 6, 4, 3, 8, 0, 4, 8, 6, 7, 2, 7, 1, 7, 4, 1, 2, 4, 2, 2, 8, 7, 2, 2, 6, 7, 5, 6, 8, 9, 9, 6, 5, 2, 9, 8, 7, 2, 3, 3, 7, 4, 3, 2, 1, 9, 0, 1, 5, 3, 2, 6, 5, 9, 1, 5, 4, 3, 6, 2, 8, 9, 7, 8, 0, 8, 9, 1, 2, 5, 7, 2, 0, 6, 6, 1, 6, 3, 1, 5, 0, 3, 7, 4, 6, 5, 6, 4, 8, 3, 4, 5, 5, 0, 6, 6, 7, 5, 0, 9, 9, 6, 1, 0, 4, 6, 3, 6, 9, 3, 6, 6, 8, 2, 1, 0, 1, 9, 3, 7, 4, 3, 0, 2, 7, 6, 4, 3, 0, 8, 2, 0, 0, 9, 6, 6, 6, 8, 7, 4, 1, 8, 1, 2, 2, 4, 8, 5, 2, 6, 5, 3, 9, 1, 0, 7, 2, 4, 4, 0, 0, 6, 2, 2, 4, 0, 5, 9, 7, 1, 8, 4, 5, 5, 9, 8, 5, 7, 8, 0, 9, 8, 9, 1, 6, 3, 8, 0, 3, 4, 4, 8, 4, 8, 9, 8, 6, 0, 0, 8, 2, 7, 4, 2, 5, 6, 0, 5, 8, 4, 1, 9, 0, 1, 4, 4, 8, 4, 9, 6, 0, 7, 7, 6, 8, 9, 6, 2, 0, 4, 9, 4, 9, 3, 9, 6, 6, 7, 0, 9, 7, 1, 8, 6, 0, 6, 7, 4, 1, 9, 4, 6, 7, 9, 8, 3, 9, 2, 1, 2, 7, 6, 1, 0, 0, 5, 6, 0, 4, 3, 2, 8, 8, 0, 6, 9, 5, 2, 8, 7, 0, 6, 5, 9, 7, 2, 3, 6, 9, 6, 2, 2, 4, 1, 0, 5, 0, 8, 9, 3, 5, 9, 3, 8, 1, 6, 3, 7, 5, 6, 2, 0, 2, 8, 2, 8, 7, 7, 8, 1, 0, 8, 9, 7, 0, 3, 8, 0, 5, 9, 5, 8, 4, 2, 0, 9, 2, 2, 4, 4, 9, 2, 2, 2, 5, 1, 3, 2, 0, 0, 4, 0, 6, 5, 8, 0, 5, 8, 6, 4, 8, 5, 2, 9, 7, 9, 7, 1, 0, 1, 9, 6, 9, 2, 7, 9, 4, 4, 0, 6, 2, 4, 1, 3, 7, 2, 8, 5, 9, 0, 3, 2, 3, 2, 7, 6, 3, 2, 5, 9, 0, 5, 9, 9, 8, 7, 7, 4, 8, 6, 5, 2, 3, 1, 0, 4, 1, 8, 8, 4, 9, 4, 4, 3, 3, 9, 2, 0, 1, 1, 8, 4, 4, 8, 3, 2, 9, 5, 7, 6, 2, 5, 4, 7, 3, 3, 9, 0, 1, 5, 9, 3, 7, 6, 0, 4, 2, 2, 5, 6, 3, 8, 9, 5, 6, 1, 4, 5, 4, 6, 7, 2, 1, 0, 2, 0, 4, 9, 8, 8, 9, 1, 1, 5, 0, 0, 8, 7, 1, 7, 4, 5, 4, 3, 3, 2, 0, 6, 6, 0, 1, 3, 9, 8, 3, 7, 8, 9, 4, 8, 9, 0, 9, 7, 1, 6, 2, 9, 5, 6, 9, 3, 9, 8, 7, 7, 1, 6, 5, 3, 1, 3, 1, 2, 7, 1, 8, 2, 0, 9, 7, 9, 8, 8, 6, 7, 3, 7, 1, 3, 9, 0, 9, 3, 6, 7, 2, 7, 3, 0, 5, 9, 7, 5, 5, 0, 6, 5, 1, 8, 2, 7, 5, 9, 0, 0, 0, 8, 8, 7, 3, 7, 8, 9, 3, 7, 9, 7, 8, 7, 9, 8, 5, 4, 8, 3, 7, 6, 3, 8, 2, 1, 9, 5, 7, 3, 9, 5, 5, 8, 7, 3, 5, 3, 5, 9, 7, 6, 7, 3, 6, 4, 3, 9, 4, 2, 1, 9, 6, 0, 2, 6, 7, 4, 7, 9, 0, 7, 4, 3, 5, 3, 1, 1, 2, 6, 8, 2, 1, 7, 8, 5, 9, 6, 1, 1, 5, 0, 6, 0, 9, 2, 6, 5, 8, 9, 5, 5, 6, 2, 9, 1, 5, 8, 8, 7, 1, 7, 3, 5, 4, 9, 7, 5, 2, 9, 9, 4, 7, 4, 1, 3, 8, 7, 9, 0, 4, 5, 7, 5, 2, 8, 7, 6, 9, 6, 9, 3, 8, 5, 6, 6, 9, 5, 7, 8, 0, 5, 0, 7, 4, 8, 2, 5, 1, 3, 2, 2, 6, 2, 1, 7, 4, 6, 3, 1, 3, 7, 2, 1, 3, 7, 0, 8, 4, 4, 5, 7, 9, 5, 4, 3, 9, 6, 8, 2, 3, 3, 1, 6, 1, 7, 0, 3, 4, 2, 9, 4, 5, 8, 2, 7, 0, 9, 6, 8, 0, 8, 2, 8, 5, 7, 7, 2, 2, 0, 0, 0, 7, 4, 1, 6, 6, 8, 8, 9, 0, 9, 0, 1, 3, 3, 0, 9, 6, 6, 2, 6, 3, 4, 0, 8, 4, 1, 4, 0, 6, 5, 0, 9, 9, 9, 9, 1, 2, 3, 5, 4, 2, 9, 6, 0, 9, 6, 6, 8, 0, 6, 1, 4, 6, 8, 0, 5, 4, 1, 2, 0, 9, 6, 4, 2, 4, 6, 5, 9, 7, 7, 4, 6, 5, 0, 1, 9, 0, 3, 1, 9, 0, 9, 7, 8, 6, 7, 6, 8, 2, 4, 5, 3, 0, 3, 2, 1, 7, 5, 9, 3, 4, 5, 7, 1, 5, 0, 1, 1, 1, 9, 7, 5, 4, 9, 7, 8, 1, 0, 2, 8, 5, 6, 7, 0, 1, 4, 8, 4, 4, 6, 6, 5, 8, 1, 8, 4, 6, 5, 9, 2, 2, 1, 4, 9, 1, 6, 7, 2, 0, 1, 7, 6, 5, 2, 2, 5, 6, 0, 9, 0, 1, 5, 3, 3, 5, 8, 7, 5, 6, 5, 8, 0, 5, 9, 4, 6, 5, 1, 1, 0, 3, 3, 9, 4, 8, 1, 7, 7, 9, 9, 4, 3, 6, 3, 2, 8, 2, 7, 6, 7, 0, 2, 1, 2, 9, 4, 6, 9, 6, 1, 0, 1, 8, 7, 0, 0, 4, 7, 4, 2, 6, 9, 5, 9, 0, 7, 4, 5, 8, 1, 4, 7, 9, 9, 8, 8, 6, 3, 7, 0, 8, 9, 6, 2, 4, 6, 2, 9, 7, 4, 6, 8, 5, 6, 1, 3, 5, 9, 9, 1, 3, 2, 0, 3, 0, 2, 0, 7, 3, 9, 3, 5, 7, 6, 5, 9, 5, 6, 1, 2, 4, 0, 2, 7, 8, 4, 4, 0, 9, 5, 9, 2, 9, 3, 2, 4, 3, 2, 2, 8, 8, 6, 8, 1, 6, 8, 9, 8, 2, 1, 4, 9, 1, 5, 7, 1, 6, 0, 1, 5, 2, 8, 1, 1, 3, 8, 1, 3, 1, 8, 5, 1, 9, 0, 3, 0, 4, 0, 5, 1, 2, 0, 4, 9, 5, 1, 2, 2, 7, 8, 7, 2, 4, 5, 3, 4, 0, 6, 1, 5, 9, 8, 0, 2, 0, 7, 6, 0, 5, 9, 7, 5, 0, 4, 6, 0, 6, 3, 6, 8, 1, 1, 8, 9, 7, 2, 9, 0, 2, 2, 9, 0, 6, 5, 7, 7, 9, 1, 7, 9, 8, 4, 5, 0, 8, 0, 2, 5, 2, 1, 4, 4, 8, 9, 7, 8, 3, 6, 6, 0, 1, 1, 1, 8, 1, 4, 4, 0, 7, 8, 2, 1, 2, 5, 4, 6, 0, 5, 7, 4, 4, 3, 9, 5, 8, 8, 0, 8, 7, 4, 1, 8, 4, 9, 5, 4, 1, 7, 7, 7, 7, 0, 3, 8, 3, 3, 0, 5, 7, 0, 8, 0, 0, 9, 2, 2, 3, 4, 8, 2, 2, 6, 3, 3, 6, 2, 9, 4, 0, 1, 7, 5, 5, 7, 3, 0, 4, 2, 0, 7, 5, 8, 0, 8, 2, 7, 0, 3, 5, 3, 8, 3, 5, 1, 7]

# Check accuracy
i = 0
misclassified = 0

filepath = "test_submit.csv" # Path to your output file
with open(filepath, "r") as f:
    next(f) # Skip first line in file
    for line in f:
        pred = int(line.split(",")[1])
        if pred != labels[i]:
            misclassified += 1
        
        i += 1

error = misclassified / i
print("N=" + str(i) + "   error=" + str(error) + "   accuracy=" + str(1 - error))

N=10000   error=0.1875   accuracy=0.8125


## Citations

Tensorflow Tutorials:

https://www.tensorflow.org/tutorials/layers
    https://github.com/tensorflow/tensorflow/blob/r1.5/tensorflow/examples/tutorials/layers/cnn_mnist.py    

https://github.com/tensorflow/models/blob/master/tutorials/image/cifar10/cifar10_input.py (useful for data augmentation)

https://github.com/tensorflow/models/blob/master/tutorials/image/cifar10/cifar10.py (inspiration for potential new architectures)

Tensorboard Tutorial:

https://github.com/dandelionmane/tf-dev-summit-tensorboard-tutorial/blob/master/mnist.py